In [1]:
 %matplotlib notebook

### Setup

In [2]:
import numpy as np 
import time 

from scipy.spatial import KDTree
import pyvista as pv
pv.set_jupyter_backend('trame')

### PyVista Capabilities

In [3]:
pcd_pv = pv.read('../data/lidar_data/MLS_UTWENTE_super_sample.ply')

pcd_pv.plot(eye_dome_lighting=True)

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x1f24b6c0990_0" src="http://localhost:8888/trame-…

In [4]:
pcd_pv.points

pyvista_ndarray([[739.93, 206.51,  47.74],
                 [749.5 , 144.79,  35.25],
                 [743.63, 136.23,  35.24],
                 ...,
                 [767.65, 136.94,  43.3 ],
                 [780.22, 121.43,  46.18],
                 [752.51, 130.07,  35.25]], dtype=float32)

In [5]:
pcd_pv['elevation'] = pcd_pv.points[:, 2] # store variable, must agree with number of points 
pv.plot(pcd_pv, scalars = pcd_pv['elevation'])

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x1f26efb95d0_1" src="http://localhost:8888/trame-…

In [7]:
pcd_pv['random'] = pcd_pv.points[:, 0] * pcd_pv.points[:, 1]
pv.plot(pcd_pv, scalars = pcd_pv['random'], render_points_as_spheres=True, point_size=5, show_scalar_bar=False)

Widget(value='<iframe id="pyvista-jupyter_trame__template_P_0x1f2054edc90_3" src="http://localhost:8888/trame-…

### Pre-processing
KD-Tree is used to find the nearest neighbour of a point

#### KD-Tree with PyVista (not time efficient)

In [8]:
temp = pcd_pv.find_closest_point((1, 1, 0), n = 20)
print(temp)

[475457  83810 494104 129202   4976 194870 360960  18065 312244 141226
 136522 381238 469938 113614 187953 244253 151554 483604 271765  97832]


#### K-Nearest neighbor structure (SciPy)

In [9]:
tree = KDTree(pcd_pv.points)

In [10]:
dists, indices = tree.query(pcd_pv.points, k = 20)

In [11]:
neighbours = pcd_pv.points[indices]
neighbours # 20 neighbours for each of the points

pyvista_ndarray([[[739.93, 206.51,  47.74],
                  [740.36, 205.84,  48.65],
                  [740.13, 205.37,  47.25],
                  ...,
                  [743.29, 208.41,  51.96],
                  [745.22, 207.83,  49.84],
                  [743.74, 204.89,  43.39]],

                 [[749.5 , 144.79,  35.25],
                  [749.53, 144.82,  35.27],
                  [749.55, 144.76,  35.26],
                  ...,
                  [749.48, 144.69,  35.18],
                  [749.49, 144.91,  35.28],
                  [749.45, 144.91,  35.28]],

                 [[743.63, 136.23,  35.24],
                  [743.63, 136.26,  35.25],
                  [743.56, 136.26,  35.25],
                  ...,
                  [743.48, 136.29,  35.24],
                  [743.48, 136.16,  35.24],
                  [743.75, 136.11,  35.25]],

                 ...,

                 [[767.65, 136.94,  43.3 ],
                  [767.64, 136.78,  43.32],
                  [767

In [12]:
len(neighbours)

500000

Find all points within distance of r of points

In [13]:
idx_temp = tree.query_ball_point(pcd_pv.points[:10], 1)

In [22]:
len(idx_temp[4])

751

### PCA 

In [23]:
X = pcd_pv.points

In [24]:
X_mean = np.mean(X, axis=0)
X_mean

pyvista_ndarray([759.67316 , 134.4555  ,  38.171036], dtype=float32)

In [25]:
centered_X = X - X_mean

In [26]:
cov_matrix = np.cov(centered_X, rowvar=False)

In [27]:
eigen_values, eigen_vectors = np.linalg.eig(cov_matrix)

In [28]:
eigen_values

array([1505.06224494,   97.80809957,   24.68897396])

In [29]:
eigen_vectors

array([[ 0.87099665, -0.48910043, -0.04632067],
       [-0.49127086, -0.86789387, -0.07357422],
       [ 0.00421624, -0.0868389 ,  0.99621345]])

In [30]:
eigen_vectors[:, 0] # eigen vector corresponding to the first eigen value

array([ 0.87099665, -0.49127086,  0.00421624])

In [31]:
sorted_index = np.argsort(eigen_values)[::-1] # get indexes sorted by values 
sorted_eigenvalue = eigen_values[sorted_index]
sorted_eigenvectors = eigen_vectors[:, sorted_index]

Note that this is computed for the whole point cloud which may not be the best approach. So we will try to run this for locally, using neighbours

In [32]:
def PCA(cloud):
    X_mean = np.mean(cloud, axis=0)
    centered_X = cloud - X_mean
    cov_matrix = np.cov(centered_X, rowvar=False)
    eigen_values, eigen_vectors = np.linalg.eig(cov_matrix)
    sorted_index = np.argsort(eigen_values)[::-1] # get indexes sorted by values 
    sorted_eigenvalue = eigen_values[sorted_index]
    sorted_eigenvectors = eigen_vectors[:, sorted_index]
    return sorted_eigenvalue, sorted_eigenvectors

In [33]:
sel = 1 
PCA(neighbours[sel])

(array([0.00556246, 0.00346349, 0.00021667]),
 array([[ 0.11213581, -0.91545711, -0.38647618],
        [ 0.84199885,  0.29407576, -0.4522802 ],
        [ 0.5276964 , -0.27469569,  0.80379026]]))

#### Knowledge Driven Custom Search

In [34]:
tree_2D = KDTree(pcd_pv.points[:, 0:2])
idx_2D_rad = tree_2D.query_ball_point(pcd_pv.points[:, 0:2], 1)

In [35]:
selection = pcd_pv.points[idx_2D_rad[sel]]

In [36]:
selection

pyvista_ndarray([[749.5 , 144.79,  35.25],
                 [750.22, 144.71,  35.28],
                 [749.51, 144.14,  35.21],
                 ...,
                 [749.33, 144.41,  35.2 ],
                 [750.13, 144.34,  35.18],
                 [750.01, 144.49,  35.27]], dtype=float32)

Maximum and minimum height distance from the selected point

In [37]:
d_high = np.array(np.max(selection, axis=0) - pcd_pv.points[sel])[2]
d_high

np.float32(18.369999)

In [38]:
d_low = np.array(np.min(selection, axis=0) - pcd_pv.points[sel])[2]
d_low

np.float32(-0.099998474)